<a href="https://colab.research.google.com/github/solharsh/Capstone_Sentiment_Analysis/blob/master/Word2Vec_Try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install unidecode

     |████████████████████████████████| 245kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


from time import time 
from collections import defaultdict

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)


In [0]:
file = pd.read_csv("/content/drive/My Drive/Capstone Project - NLP/Harsh/speech_lemma_cust_stop.csv")

In [0]:
def text_to_word_list(text, remove_unnecessary_letters):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = remove_unnecessary_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [0]:
file.Speech_Lemma_Cust_Stop = file.Speech_Lemma_Cust_Stop.apply(lambda x: text_to_word_list(x, unidecode))

In [0]:
file_model = file.copy()
file_model = file_model[file_model.Speech_Lemma_Cust_Stop.str.len()>1]

In [12]:
sent = [row for row in file_model.Speech_Lemma_Cust_Stop]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

INFO - 10:38:16: collecting all words and their counts
INFO - 10:38:16: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 10:38:16: collected 9748 word types from a corpus of 11538 words (unigram + bigrams) and 12 sentences
INFO - 10:38:16: using 9748 counts as vocab in Phrases<0 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 10:38:16: source_vocab length 9748
INFO - 10:38:16: Phraser built with 533 phrasegrams


['minister',
 'decisively',
 'status',
 'quo',
 'poverty_poverty',
 'class',
 'society',
 'decision',
 'loss',
 'lower',
 'concern',
 'presently',
 'recovery_recovery',
 'vis_a',
 'vis',
 'recovery',
 'minister',
 'macro_economic',
 'lower',
 'external',
 'comprehensive',
 'macro_economic',
 'lower',
 'deficit',
 'current_deficit',
 'expect',
 'poverty',
 'anti',
 'poverty',
 'reflected',
 'minister',
 'sab',
 'vikas',
 'made',
 'response',
 'inter_generational',
 'debt',
 'date',
 'non',
 'tax',
 'decline',
 'deficit_mainly',
 'reduction',
 'external',
 'turn',
 'around',
 'ending',
 'current_deficit',
 'mainly',
 'non_essential',
 'demand',
 'predecessor',
 'reducing',
 'deficit',
 'current',
 'low',
 'deficit',
 'may',
 'decided',
 'deficit',
 'middle',
 'east',
 'recently',
 'problem',
 'minimum_maximum',
 'governance_operational',
 'maximum',
 'report',
 'sc/sts',
 'scheme',
 'power',
 'keeping_mind',
 'honble',
 'consequent',
 'conclusion',
 'juncture',
 'predictable',
 'keeping_

min count = 3 - remove most unusual words from training embeddings, like words 'promote_entrepreneurship', which actually stands for 'promote entrepreneaurship', and doesn't need additional training

- window = 4 - Word2Vec model will learn to predict given word from up to 4 words to the left, and up to 4 words to the right

- size = 300 - size of hidden layer used to predict surroundings of embedded word, which also stands for dimensions of trained embeddings

- sample = 1e-5 - probability baseline for subsampling most frequent words from surrounding of embedded word

- negative = 20 - number of negative (ones that shouldn't have been predicted while modeling selected pair of words) words that will have their corresponding weights updated while training on specific training example, along with positive word

In [13]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 10:39:43: collecting all words and their counts
INFO - 10:39:43: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 10:39:43: collected 1948 word types from a corpus of 9936 raw words and 12 sentences
INFO - 10:39:43: Loading a fresh vocabulary
INFO - 10:39:43: effective_min_count=3 retains 545 unique words (27% of original 1948, drops 1403)
INFO - 10:39:43: effective_min_count=3 leaves 8124 word corpus (81% of original 9936, drops 1812)
INFO - 10:39:43: deleting the raw counts dictionary of 1948 items
INFO - 10:39:43: sample=1e-05 downsamples 545 most-common words
INFO - 10:39:43: downsampling leaves estimated 535 word corpus (6.6% of prior 8124)
INFO - 10:39:43: estimated required memory for 545 words and 300 dimensions: 1580500 bytes
INFO - 10:39:43: resetting layer weights


Time to build vocab: 0.0 mins


In [14]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 10:39:53: training model with 1 workers on 545 vocabulary and 300 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4
INFO - 10:39:53: worker thread finished; awaiting finish of 0 more threads
INFO - 10:39:53: EPOCH - 1 : training on 9936 raw words (499 effective words) took 0.0s, 14276 effective words/s
INFO - 10:39:53: worker thread finished; awaiting finish of 0 more threads
INFO - 10:39:53: EPOCH - 2 : training on 9936 raw words (543 effective words) took 0.0s, 16513 effective words/s
INFO - 10:39:53: worker thread finished; awaiting finish of 0 more threads
INFO - 10:39:53: EPOCH - 3 : training on 9936 raw words (548 effective words) took 0.0s, 19831 effective words/s
INFO - 10:39:53: worker thread finished; awaiting finish of 0 more threads
INFO - 10:39:53: EPOCH - 4 : training on 9936 raw words (506 effective words) took 0.0s, 18827 effective words/s
INFO - 10:39:53: worker thread finished; awaiting finish of 0 more threads
INFO - 10:39:53: EPOCH - 5 : training on

Time to train the model: 0.02 mins


In [15]:
w2v_model.save("word2vec_another.model")

INFO - 10:40:09: saving Word2Vec object under word2vec_another.model, separately None
INFO - 10:40:09: not storing attribute vectors_norm
INFO - 10:40:09: not storing attribute cum_table
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
INFO - 10:40:09: saved word2vec_another.model


In [0]:
file_export = file_model.copy()
file_export['Speech_Lemma_Cust_Stop'] = file_export.Speech_Lemma_Cust_Stop
file_export.Speech_Lemma_Cust_Stop = file_export.Speech_Lemma_Cust_Stop.apply(lambda x: ' '.join(bigram[x]))

In [0]:
file_export.to_csv('cleaned_dataset.csv', index=False)

In [20]:
word_vectors = Word2Vec.load("/content/word2vec_another.model").wv


INFO - 10:42:13: loading Word2Vec object from /content/word2vec_another.model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
INFO - 10:42:13: loading wv recursively from /content/word2vec_another.model.wv.* with mmap=None
INFO - 10:42:13: setting ignored attribute vectors_norm to None
INFO - 10:42:13: loading vocabulary recursively from /content/word2vec_another.model.vocabulary.* with mmap=None
INFO - 10:42:13: loading trainables recursively from /content/word2vec_another.model.trainables.* with mmap=None
INFO - 10:42:13: setting ignored attribute cum_table to None
INFO - 10:42:13: loaded /content/word2vec_another.model


In [0]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors)

In [23]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

INFO - 10:42:36: precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('super', 0.9999998807907104),
 ('lock_in', 0.9994293451309204),
 ('non_performing', 0.9994256496429443),
 ('e', 0.9994200468063354),
 ('external', 0.9994189739227295),
 ('class', 0.9994183778762817),
 ('private', 0.9994181394577026),
 ('produce', 0.9994180202484131),
 ('film', 0.9994176626205444),
 ('sum', 0.9994176030158997)]

In [0]:
positive_cluster_center = model.cluster_centers_[0]
negative_cluster_center = model.cluster_centers_[1]

In [29]:
words = pd.DataFrame(word_vectors.vocab.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors.wv[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
words['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  
INFO - 10:43:39: NumExpr defaulting to 2 threads.


In [31]:
words.head(10)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,minister,"[-0.00030517866, -0.07556432, -0.0051926235, -...",1,-1,239.548713,-239.548713
1,recovery,"[0.00058132334, -0.0756106, -0.005140628, -0.0...",1,-1,184.170159,-184.170159
2,debt,"[0.0002607753, -0.075390615, -0.0053126127, -0...",1,-1,183.606724,-183.606724
3,ones,"[0.0014946946, -0.07712085, -0.0033782392, -0....",1,-1,66.317436,-66.317436
4,indias,"[0.00039181928, -0.07590666, -0.004629179, -0....",1,-1,269.540627,-269.540627
5,product,"[-0.0005033336, -0.076035015, -0.0046590795, -...",1,-1,182.178554,-182.178554
6,preceding,"[0.0006061077, -0.07523416, -0.0047281403, -0....",1,-1,150.709033,-150.709033
7,years,"[0.0012034712, -0.074813545, -0.006155806, -0....",1,-1,73.126230,-73.126230
8,response,"[-0.0006459907, -0.07620716, -0.004064309, -0....",1,-1,109.224104,-109.224104
9,expanded,"[0.0003275104, -0.076102555, -0.005201346, -0....",1,-1,166.405656,-166.405656


In [53]:
words.tail(10)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
535,sub_plan,"[-0.00039711164, -0.07526852, -0.0048797587, -...",1,-1,148.799847,-148.799847
536,across,"[-6.605081e-05, -0.076037645, -0.004618882, -0...",1,-1,167.555526,-167.555526
537,w_e,"[-0.00051648536, -0.07572093, -0.005264799, -0...",1,-1,80.940936,-80.940936
538,f,"[0.00013480373, -0.07438007, -0.0051311315, -0...",1,-1,66.701085,-66.701085
539,charger/adapter_headsets/speakers,"[0.00074677466, -0.07657748, -0.005479288, -0....",1,-1,99.961028,-99.961028
540,set_top,"[-0.0006023316, -0.075583525, -0.004217541, -0...",1,-1,149.250401,-149.250401
541,repair_condition,"[0.0006244523, -0.075133726, -0.0052383034, -0...",1,-1,116.650753,-116.650753
542,pile_epingle,"[0.00068346446, -0.07575631, -0.0057524047, -0...",1,-1,106.838911,-106.838911
543,demand_supply,"[0.0001709611, -0.07566928, -0.003781176, -0.0...",1,-1,73.223686,-73.223686
544,extend_concession,"[-0.0007263941, -0.07511506, -0.004727404, -0....",1,-1,137.519275,-137.519275


In [0]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)